# AutoPrompting with datasets

In [ ]:
from datasets import load_dataset

sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:100]
gen_targets = samsum['train']['summary'][:100]

Starting with PromptTuner

In [ ]:
from coolprompt.assistant import PromptTuner

# Define an initial prompt
class_start_prompt = 'Perform Sentiment Classification task.'

# Initialize the tuner
tuner = PromptTuner()

# Call prompt optimization with dataset and target
final_prompt = tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="accuracy"
)

You can now get initial and final prompt metrics from tuner fields

In [ ]:
print("Final prompt:", final_prompt)
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)

You can do the same with generation task

Also you can reuse previous tuner binded with LLM

In [ ]:
gen_start_prompt = "Summarize the text"

final_prompt = tuner.run(
    start_prompt=gen_start_prompt,
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="meteor"
)

print("Final prompt:", final_prompt)
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)

Currently supported metrics are
- accuracy and f1 for classification
- meteor, bleu and rouge for generation

Also, task type must correspond the metric

In [ ]:
tuner.run(
    start_prompt=class_start_prompt,
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="rouge"
)

There are two ways to initialize tuner with your custom LLM

To init a model by yourself and pass it to the tuner

In [ ]:
from langchain_community.llms import VLLM

my_model = VLLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True,
    dtype='float16',
)

tuner_with_custom_llm = PromptTuner(model=my_model)
tuner_with_custom_llm.run(start_prompt="Write an essay about autumn")

Or to change config of our default model

In [ ]:
from coolprompt.language_model.llm import DefaultLLM

changed_model = DefaultLLM.init(langchain_config={
    'max_new_tokens': 1000,
    "temperature": 0.0,
})

tuner_with_changed_llm = PromptTuner(model=changed_model)
tuner_with_changed_llm.run(start_prompt="Write an essay about autumn")

You can access prompts and their metrics via tuner fields

In [ ]:
def print_results(tuner):
    print("Start prompt:", tuner.start_prompt)
    print("Final prompt:", tuner.final_prompt)
    print("Start prompt metric: ", tuner.init_metric)
    print("Final prompt metric: ", tuner.final_metric)

There are 3 currently implemented optimizers:
- HyPE (optimizing with predefined system instruction)
- DistillPrompt
- ReflectivePrompt

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task="classification",
    method="hype",
    dataset=class_dataset,
    target=class_targets,
    metric="accuracy"
)

print_results(tuner)

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='distill',
    use_cache=True,
    num_epochs=1
)

print_results(tuner)

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='reflective',
    problem_description='sentiment classification',
    use_cache=False,
    population_size=4,
    num_epochs=3
)

print_results(tuner)